In [ ]:
%sql
USE CATALOG agilisium_playground;

-- Databricks SQL script: Inventory at Risk calculation and validation suite for agilisium_playground.purgo_playground.f_inv_movmnt
-- Purpose: Implements total and grouped inventory at risk summaries; validates enumeration, edge conditions, and type-correctness
-- Author: Sang Kim
-- Date: 2025-07-29
-- Description: 
--   Provides inventory-at-risk summaries and validation per AD-310. 
--   Applies CTEs, proper column comments, precise filters, and returns results 
--   only for financial_qty where flag_active = 'Y' and value is non-null, as specified.

-- ===========================================================================
/* COLUMN DESCRIPTIONS AND ENUM DOCUMENTATION (as inline comments, not DDL) */
-- ===========================================================================

-- financial_qty: Financial inventory quantity at risk (DOUBLE). Only non-null values are included in the aggregation.
-- flag_active: Active flag. Valid values: 'Y' = active/included, 'N' = inactive/excluded. Case sensitive; other values are ignored.
-- inv_loc: Inventory location code (STRING).
-- item_nbr: Item number identifier (STRING).
-- plant_loc_cd: Plant/location code (STRING).

-- ===========================================================================
/* CTE: TOTAL INVENTORY AT RISK FOR "ACTIVE" RECORDS */
-- ===========================================================================

-- Sums financial_qty where flag_active = 'Y' and financial_qty IS NOT NULL.
WITH cte_at_risk_total AS (
  SELECT
    SUM(financial_qty) AS financial_qty_at_risk
  FROM agilisium_playground.purgo_playground.f_inv_movmnt
  WHERE flag_active = 'Y'
    AND financial_qty IS NOT NULL
)
SELECT financial_qty_at_risk
FROM cte_at_risk_total
;

-- ===========================================================================
/* CTE: GROUPED INVENTORY AT RISK BY inv_loc, item_nbr, plant_loc_cd */
-- ===========================================================================

-- Sums financial_qty where flag_active = 'Y' and financial_qty IS NOT NULL, grouped by inv_loc, item_nbr, plant_loc_cd.
WITH cte_inventory_at_risk_grouped AS (
  SELECT
    inv_loc,
    item_nbr,
    plant_loc_cd,
    SUM(financial_qty) AS financial_qty_at_risk
  FROM agilisium_playground.purgo_playground.f_inv_movmnt
  WHERE flag_active = 'Y'
    AND financial_qty IS NOT NULL
  GROUP BY inv_loc, item_nbr, plant_loc_cd
)
SELECT
  inv_loc,
  item_nbr,
  plant_loc_cd,
  financial_qty_at_risk
FROM cte_inventory_at_risk_grouped
;

-- ===========================================================================
/* CTE: RESULT FOR SCENARIO WHERE THERE ARE NO ACTIVE RECORDS */
-- ===========================================================================

-- Returns null or zero if there are no records with flag_active = 'Y'.
WITH cte_none_active AS (
  SELECT
    SUM(financial_qty) AS financial_qty_at_risk
  FROM agilisium_playground.purgo_playground.f_inv_movmnt
  WHERE flag_active = 'NO_ACTIVE_FLAG'
)
SELECT financial_qty_at_risk
FROM cte_none_active
;

-- ===========================================================================
/* CTE: VALIDATION - ENUMERATION & TYPE ENFORCEMENT FOR FLAG_ACTIVE */
-- ===========================================================================

-- Sums by flag_active value variants and checks filtering correctness.
WITH cte_flag_sum AS (
  SELECT
    SUM(CASE WHEN flag_active = 'Y' AND financial_qty IS NOT NULL THEN financial_qty ELSE 0 END) AS active_sum,
    SUM(CASE WHEN flag_active = 'N' AND financial_qty IS NOT NULL THEN financial_qty ELSE 0 END) AS inactive_sum,
    SUM(CASE WHEN flag_active IS NULL AND financial_qty IS NOT NULL THEN financial_qty ELSE 0 END) AS null_flag_sum,
    SUM(CASE WHEN TRIM(flag_active) = '' AND financial_qty IS NOT NULL THEN financial_qty ELSE 0 END) AS empty_flag_sum,
    SUM(CASE WHEN flag_active = 'y' AND financial_qty IS NOT NULL THEN financial_qty ELSE 0 END) AS lower_y_flag_sum
  FROM agilisium_playground.purgo_playground.f_inv_movmnt
)
SELECT * FROM cte_flag_sum
;

-- ===========================================================================
/* CTE: MIN/MAX/ZERO/NEGATIVE CHECK FOR ACTIVE, NON-NULL financial_qty */
-- ===========================================================================

-- Gives min/max, zero count, and negative value count for aggregation edge testing.
WITH cte_extremes AS (
  SELECT
    MIN(financial_qty) AS min_value,
    MAX(financial_qty) AS max_value,
    SUM(CASE WHEN financial_qty = 0.0 THEN 1 ELSE 0 END) AS zero_count,
    SUM(CASE WHEN financial_qty < 0 THEN 1 ELSE 0 END) AS neg_count
  FROM agilisium_playground.purgo_playground.f_inv_movmnt
  WHERE flag_active = 'Y' AND financial_qty IS NOT NULL
)
SELECT * FROM cte_extremes
;

-- ===========================================================================
/* End of Databricks SQL implementation for Inventory at Risk summary and validation. */
-- ===========================================================================
